In [2]:
import json
from dotenv import load_dotenv
import concurrent.futures
import os, time
import tempfile
import promptflow as pf

In [2]:
# %pip install azure-ai-generative[evaluate]
#!pip install azure-ai-generative
#!pip install azure-ai-ml

In [ ]:
C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence\src\evaluators\flow.dag.yaml

In [3]:
EVAL_FLOW = "C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-document-intelligence\\src\\evaluators\\flow.dag.yaml"
chat_history = []
question = "What is the best way to manufacture a car?"
answer = "audi"
context = "because is better than bmw"

In [4]:
cli = pf.PFClient()
result = cli.test(
    EVAL_FLOW,
    inputs=dict(
        chat_history=chat_history, question=question, answer=answer, context=context
    ),
)
print("result:", result)

2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.
2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Executing node coherence_score. node run id: 210100c1-f9e7-4b35-b78e-ecc099781195_coherence_score_0
2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Executing node fluency_score. node run id: 210100c1-f9e7-4b35-b78e-ecc099781195_fluency_score_0
2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Executing node groundedness_score. node run id: 210100c1-f9e7-4b35-b78e-ecc099781195_groundedness_score_0
2024-01-20 20:38:19 -0600   45416 execution.flow     INFO     Executing node relevance_score. node run id: 210100c1-f9e7-4b35-b78e-ecc099781195_relevance_score_0
2024-01-20 20:38:20 -0600   45416 execution.flow     INFO     Node fluency_score completes.
2024-01-20 20:38:20 -0600   45416 execution

In [18]:
from typing import List, Any


def call_pf_client_and_test(
    eval_flow: str, chat_history: List[str], question: str, answer: str, context: str
) -> Any:
    """
    Creates an instance of PFClient, calls the test method with provided arguments, and returns the result.

    Parameters:
    eval_flow (str): The path to the evaluation flow file.
    chat_history (List[str]): The chat history to be passed to the test method.
    question (str): The question to be passed to the test method.
    answer (str): The answer to be passed to the test method.
    context (str): The context to be passed to the test method.

    Returns:
    Any: The result returned from the test method of PFClient.
    """

    # Create an instance of PFClient
    cli = pf.PFClient()

    # Call the test method and return the result
    return cli.test(
        eval_flow,
        inputs=dict(
            chat_history=chat_history, question=question, answer=answer, context=context
        ),
    )

In [19]:
import json
import logging
from typing import List

# Set up logger
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)


def extract_conversation_data(file_path: str, eval_flow: str) -> None:
    """
    Extracts and logs questions, user responses, and context from a newline-delimited JSON file.
    Calls the call_pf_client_and_test function for each line and collects results in a summary table.

    Parameters:
    file_path (str): The path to the newline-delimited JSON file.
    eval_flow (str): The path to the evaluation flow file.

    Returns:
    None: This function does not return anything. It prints a summary of results.
    """

    results = []

    try:
        with open(file_path, "r") as file:
            for line in file:
                try:
                    data = json.loads(line.strip())
                    chat_history = []
                    question = ""
                    answer = ""
                    context = ""

                    for item in data.get("messages", []):
                        role = item.get("role", "")
                        content = item.get("content", "")
                        if role == "system":
                            question = content
                        elif role == "user":
                            answer = content
                        elif role == "context":
                            context = content

                    if question and answer and context:
                        result = call_pf_client_and_test(
                            eval_flow, chat_history, question, answer, context
                        )
                        results.append((question, answer, context, result))

                except json.JSONDecodeError as e:
                    logger.error("JSONDecodeError while parsing line: %s", e)

    except Exception as e:
        logger.error("An error occurred: %s", e)
        raise

    # Print summary table
    print("\nSummary of Results:")
    print("Question | Answer | Context | Result")
    print("-" * 50)
    for q, a, c, r in results:
        print(f"{q} | {a} | {c} | {r}")

In [20]:
extract_conversation_data(
    file_path="C:\\Users\\pablosal\\Desktop\\azure-ai-gbb-solutions\\workshop\\solution\\build_your_own_copilot_aoai\\rag_pattern\\manufacturing_bot\\evaluation\\bulk\\validation.jsonl",
    eval_flow=EVAL_FLOW,
)

2023-11-28 19:43:16 -0600    9828 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.
2023-11-28 19:43:16 -0600    9828 execution.flow     INFO     Executing node coherence_score. node run id: 9cc148df-bdff-4dd9-a490-204a7deb7b72_coherence_score_0
2023-11-28 19:43:16 -0600    9828 execution.flow     INFO     Executing node fluency_score. node run id: 9cc148df-bdff-4dd9-a490-204a7deb7b72_fluency_score_0
2023-11-28 19:43:16 -0600    9828 execution.flow     INFO     Executing node groundedness_score. node run id: 9cc148df-bdff-4dd9-a490-204a7deb7b72_groundedness_score_0
2023-11-28 19:43:16 -0600    9828 execution.flow     INFO     Executing node relevance_score. node run id: 9cc148df-bdff-4dd9-a490-204a7deb7b72_relevance_score_0
2023-11-28 19:43:17 -0600    9828 execution          WARNING  [fluency_score in line 0 (index starts from 0)] stderr> Exception occurs: RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-

: 